## The battle of the neighborhoods

<p> Lets get out the information </p>

In [1]:
wikipedia_link="https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"

In [2]:
import requests
content_page=requests.get(wikipedia_link)

<p>Create a beautifulsoup object </br> From this object <b> I reach to the table-tag </b>  (print is commented out, you can use it if you like). </p>

In [3]:
from bs4 import BeautifulSoup
page =BeautifulSoup(content_page.text, "lxml")
table = page.table

<p style="color:red"> With the table object, we reach to all entries with find_all  <p>

<p style="font-size:120%; color:red">From all of the table we need the body (tag "tr"), and then we need the headers (tag "th").<p>

In [4]:
table_content=table.tbody
entries = table_content.find_all("tr")
headers=[];
for col in table.find_all("th"):
    headers.append(col.text)
headers[-1]

'Neighbourhood\n'

<p> The last header includes a "\n" due to line break.<\br> We need to remove it <p>
    <p style="font-size:160%; color:red"> Create a PANDA dataframe with those headers <p>

In [5]:
headers[-1]=headers[-1].rstrip()

import pandas as pd
data_canada=pd.DataFrame(columns=headers)
data_canada.columns

Index(['Postcode', 'Borough', 'Neighbourhood'], dtype='object')

<p style="font-size:120%; color:red"> Now fill the dataframe with every entry in the table (tag "td")<p>
    <div><i> Look at the comments <i/><div/>

In [6]:
if not data_canada.empty:# I run this code many times and use append(). 
                         #I need to empty df for every time I run it. 
    data_canada.drop(data_canada.index, inplace=True) 
    print("Empty dataframe")
    
for entry in entries:# Go through the body of the table object.
    try:
        row=entry.find_all("td")# Every row
        row = [x.text for x in row]# Solution from stackoverflow   
        if row[2] == "Not assigned\n":
            row[2] = row[1]+"\n" # Assign name of Borough to Neigh. if Neigh is not assigned
        s = pd.Series(row, index=data_canada.columns)# Lists cannot become rows. Series yes
        data_canada = data_canada.append(s, ignore_index=True)
    except:
        pass
    
# Remove '\n'. Use "headers" list to keep it gral. for any table we find
data_canada[headers[-1]]=data_canada[headers[-1]].apply(lambda x: x.rstrip())
# Remove rows without Borough
data_canada = data_canada[data_canada.Borough != "Not assigned"]
data_canada.head()

,Postcode,Borough,Neighbourhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront
5,M5A,Downtown Toronto,Regent Park
6,M6A,North York,Lawrence Heights


In [7]:
post_can = pd.DataFrame(columns=headers) # New dataframe
grouped = data_canada.groupby(["Postcode", "Borough"])["Neighbourhood"].apply(", ".join)

post_can = grouped.to_frame().reset_index() # Fill the dataframe with content of groupby-object
print("\n Number of Neighbourhoods are {} and you have {} Postcodes \n".format(data_canada.shape[0], grouped.count()))

#Save to file
import csv
post_can.to_csv('post_can_file.csv', index = None, header=True)

post_can.head()


 Number of Neighbourhoods are 211 and you have 103 Postcodes 



,Postcode,Borough,Neighbourhood
0,M1B,Scarborough,"Rouge, Malvern"
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae
